In [1]:
import numpy as np
import pandas as pd
from surprise import SVD
from surprise import Dataset
from surprise import Reader
from surprise import accuracy
from surprise import BaselineOnly
from surprise.model_selection import train_test_split

In [5]:
# 6 millions rows
ratings = pd.read_csv('ratings.csv')
ratings['user_id'] = ratings['user_id'].astype(str)
ratings.head()


,user_id,book_id,rating
0,1,258,5
1,2,4081,4
2,2,260,5
3,2,9296,5
4,2,2318,3


In [7]:
# 10k books
books = pd.read_csv('books.csv')
books.columns

Index(['book_id', 'goodreads_book_id', 'best_book_id', 'work_id',
       'books_count', 'isbn', 'isbn13', 'authors', 'original_publication_year',
       'original_title', 'title', 'language_code', 'average_rating',
       'ratings_count', 'work_ratings_count', 'work_text_reviews_count',
       'ratings_1', 'ratings_2', 'ratings_3', 'ratings_4', 'ratings_5',
       'image_url', 'small_image_url'],
      dtype='object')

In [9]:
# context
tags = pd.read_csv('tags.csv')
tags

,tag_id,tag_name
0,0,-
1,1,--1-
2,2,--10-
3,3,--12-
4,4,--122-
...,...,...
34247,34247,Ｃhildrens
34248,34248,Ｆａｖｏｒｉｔｅｓ
34249,34249,Ｍａｎｇａ
34250,34250,ＳＥＲＩＥＳ


In [ ]:
# create a mapping between book_id and name
id_to_name = {}
for row in new_books.itertuples():
    id_to_name[row[1]] = row[11]

# add my onw book ratings
my_rating = {'user_id': [str(53425) for i in range(11)],
             'book_id': [13, 119, 240, 283, 479, 1100, 2205, 2246, 3227, 7210, 5],
             'rating': [5, 3, 4, 4, 4, 5, 2, 4, 4, 3, 3]}
print([id_to_name[id] for id in [13, 119, 240, 283, 479, 1100, 2205, 2246, 3227, 7210, 5]])
full_rating = pd.DataFrame(my_rating).append(new_rating) 